<a href="https://colab.research.google.com/github/kunalsonalkar/transformers-nlp/blob/main/05_TinyVGG_Going_Modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import os
import zipfile

from pathlib import Path

import requests
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
  print(f"{image_path} directory exists")
else:
  print(f"Did not find {image_path} directory, creating one..")
  image_path.mkdir(parents=True, exist_ok=True)

with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading the pizza, steak and sushi data...")
  f.write(request.content)

with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzippping the image data")
  zip_ref.extractall(image_path)

# remove the zip file
os.remove(data_path/"pizza_steak_sushi.zip")

data/pizza_steak_sushi directory exists
Unzippping the image data


In [24]:
train_dir = image_path /"train"
test_dir = image_path /"test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

# Creating datasets

In [25]:
from torchvision import datasets, transforms

data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

train_data = datasets.ImageFolder(root=train_dir,
                                  transform=data_transform,
                                  target_transform=None)
test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

print(f"Train_data is {train_data}")

print(f"Test_data is {test_data}")

Train_data is Dataset ImageFolder
    Number of datapoints: 225
    Root location: data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )
Test_data is Dataset ImageFolder
    Number of datapoints: 75
    Root location: data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


In [26]:
# Getting class names
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [27]:
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

# Creating dataloaders

In [28]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=1,
                              num_workers=1,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=1,
                             num_workers=1,
                             shuffle=False)

train_dataloader,test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7f77b1cd95d0>,
 <torch.utils.data.dataloader.DataLoader at 0x7f77b1cda3e0>)

In [29]:
img, label = next(iter(train_dataloader))
print(f"Image shape: {img.shape}")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([1, 3, 64, 64])
Label shape: torch.Size([1])


In [30]:
img

tensor([[[[0.9333, 0.9216, 0.8941,  ..., 0.9098, 0.9412, 0.9490],
          [0.9294, 0.6863, 0.4314,  ..., 0.5608, 0.8941, 0.9451],
          [0.8824, 0.4235, 0.7098,  ..., 0.1765, 0.7451, 0.9451],
          ...,
          [0.8431, 0.2588, 0.0392,  ..., 0.5333, 0.6157, 0.9451],
          [0.8588, 0.2784, 0.0431,  ..., 0.5451, 0.6745, 0.9490],
          [0.8902, 0.3529, 0.0471,  ..., 0.5412, 0.7490, 0.9490]],

         [[0.9333, 0.9216, 0.8980,  ..., 0.9137, 0.9412, 0.9490],
          [0.9294, 0.6745, 0.4275,  ..., 0.5608, 0.8941, 0.9451],
          [0.8824, 0.4196, 0.7255,  ..., 0.1843, 0.7451, 0.9451],
          ...,
          [0.8392, 0.2431, 0.0314,  ..., 0.5490, 0.6196, 0.9529],
          [0.8588, 0.2667, 0.0392,  ..., 0.5608, 0.6745, 0.9529],
          [0.8863, 0.3412, 0.0392,  ..., 0.5529, 0.7451, 0.9529]],

         [[0.9255, 0.9137, 0.8824,  ..., 0.9137, 0.9412, 0.9490],
          [0.9216, 0.6706, 0.4235,  ..., 0.5569, 0.8941, 0.9490],
          [0.8784, 0.4275, 0.7255,  ..., 0

#Creating a script mode

In [31]:
import os
os.makedirs("modular_scripts")

FileExistsError: ignored

In [32]:
NUM_WORKERS = os.cpu_count()

In [33]:
%%writefile modular_scripts/data_setup.py
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
  train_dir: str,
  test_dir: str,
  transform: transforms.Compose,
  batch_size: int,
  num_workers: int=NUM_WORKERS
):

  """
  train_dir: Training directory path
  test_dir: Testing directory path
  transform: The transformations we want to do over data
  batch_size: standard batch size
  """
  train_data = datasets.ImageFolder(root=train_dir,
                                    transform=transform,
                                    target_transform=None)
  test_data = datasets.ImageFolder(root=test_dir,
                                  transform=transform)
  class_names = train_data.classes

  train_dataloader = DataLoader(dataset=train_data,
                                batch_size=batch_size,
                                num_workers=num_workers,
                                shuffle=True)
  test_dataloader = DataLoader(dataset=test_data,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              shuffle=False)
  return train_dataloader, test_dataloader, class_names


Overwriting modular_scripts/data_setup.py


Calling the dataloaders from data_setup.py

In [34]:
from modular_scripts import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir,
                                                                               test_dir,
                                                                               transform=data_transform,
                                                                               batch_size=1,
                                                                               num_workers=NUM_WORKERS)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f78780115a0>,
 ['pizza', 'steak', 'sushi'])

# Defining TinyVGG

In [35]:
import torch

from torch import nn

class TinyVGG01(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )


  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

In [36]:
model_01 = TinyVGG01(input_shape = 3,
                    hidden_units = 10,
                    output_shape=len(train_data.classes))
model_01

TinyVGG01(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [37]:
# Testing the model with single forward pass
img_batch, label_batch = next(iter(train_dataloader))

img_single, label_single = img_batch[0].unsqueeze(dim=0),label_batch[0]
print(f"Single image shape: {img_single.shape}\n")
model_01.eval()
with torch.inference_mode():
  pred = model_01(img_single)

print(f"Output logits: {pred} \n")
print(f"Output prediction: {torch.softmax(pred, dim=1)}\n")
print(f"Prediction label: {torch.argmax(torch.softmax(pred, dim=1),dim=1)}\n")
print(f"Actual label: {label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits: tensor([[-0.0007,  0.0266, -0.0318]]) 

Output prediction: tensor([[0.3337, 0.3429, 0.3235]])

Prediction label: tensor([1])

Actual label: 1


# Making a modular script from TinyVGG

In [38]:
%%writefile modular_scripts/model_builder.py
import torch

from torch import nn
class TinyVGG01(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )


  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

Overwriting modular_scripts/model_builder.py


In [39]:
from modular_scripts import model_builder

In [40]:
torch.manual_seed(42)
model_02 = model_builder.TinyVGG01(input_shape=3,
                                   hidden_units=10,
                                   output_shape=len(class_names))
model_02

TinyVGG01(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [41]:
img_batch, label_batch = next(iter(train_dataloader))
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]

print(f"Single image shape: {img_single.shape}")
model_02.eval()
with torch.inference_mode():
  pred = model_02(img_single)

print(f"output logits: {pred} \n")
print(f"Prediciton probabilities: {torch.softmax(pred, dim = 1)} \n")
print(f"Output prediction label: {torch.argmax(torch.softmax(pred, dim =1),dim =1)}")

Single image shape: torch.Size([1, 3, 64, 64])
output logits: tensor([[ 0.0208, -0.0020,  0.0095]]) 

Prediciton probabilities: tensor([[0.3371, 0.3295, 0.3333]]) 

Output prediction label: tensor([0])


#Create train_step and test_step function

In [64]:
from typing import Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
  model.train()

  train_loss, train_acc = 0,0

  for batch, (X, y) in enumerate(dataloader):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim =1), dim =1)
    train_acc += (y_pred_class==y).sum().item()/len(y_pred)
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc


In [52]:
# test_step function

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
  model.eval()
  test_loss, test_acc = 0,0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      test_pred = model(X)
      loss = loss_fn(test_pred, y)
      test_loss += loss.item()

      y_test_class = torch.argmax(torch.softmax(test_pred, dim=1),dim=1)
      test_acc += (y_test_class == y).sum().item()/len(test_pred)

  test_loss = test_loss/len(dataloader)
  test_acc = test_acc/len(dataloader)
  return test_loss, test_acc

In [46]:
# combining the train_step and test_step function

from typing import Dict, List

from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int) -> Dict[str, List[float]]:
  results = {"train_loss":[],
             "train_acc":[],
             "test_loss":[],
             "test_acc":[]
             }
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer)
    test_loss, test_acc = test_step(model, test_dataloader, loss_fn)
    print(f"Epoch: {epoch+1} \n")
    print(f"train_loss: {train_loss} \n")
    print(f"test_loss: {test_loss} \n")
    print(f"train_accuracy: {train_acc}")

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)
  return results



# Turning the previous steps into a modular script

In [65]:
%%writefile modular_scripts/engine.py

from typing import Dict, List, Tuple

from tqdm.auto import tqdm

import torch

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
  model.train()

  train_loss, train_acc = 0,0

  for batch, (X, y) in enumerate(dataloader):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim =1), dim =1)
    train_acc += (y_pred_class==y).sum().item()/len(y_pred)
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
  model.eval()
  test_loss, test_acc = 0,0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      test_pred = model(X)
      loss = loss_fn(test_pred, y)
      test_loss += loss.item()

      y_test_class = torch.argmax(torch.softmax(test_pred, dim=1),dim=1)
      test_acc += (y_test_class == y).sum().item()/len(test_pred)

  test_loss = test_loss/len(dataloader)
  test_acc = test_acc/len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int) -> Dict[str, List[float]]:
  results = {"train_loss":[],
             "train_acc":[],
             "test_loss":[],
             "test_acc":[]
             }
  for epoch in range(epochs):
    train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer)
    test_loss, test_acc = test_step(model, test_dataloader, loss_fn)
    print(f"Epoch: {epoch+1} \n")
    print(f"train_loss: {train_loss} \n")
    print(f"test_loss: {test_loss} \n")
    print(f"train_accuracy: {train_acc}")

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)
  return results


Overwriting modular_scripts/engine.py


# Creating and saving the model

In [49]:
%%writefile modular_scripts/utils.py

import torch
from pathlib import Path


def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,exist_ok=True)

  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model name should end with .pt or .pth"
  model_save_path = target_dir_path / model_name

  print(f"Saving the model to {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing modular_scripts/utils.py


# Train, Evaluate and Save the model

In [53]:
from re import I
torch.manual_seed(42)

NUM_EPOCHS = 3

model_03 = TinyVGG01(input_shape=3,
                     hidden_units=10,
                     output_shape=len(class_names))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_03.parameters(), lr=0.001)

model_03_results = train(model=model_03,
                         train_dataloader=train_dataloader,
                         test_dataloader=test_dataloader,
                         optimizer=optimizer,
                         loss_fn=loss_fn,
                         epochs=NUM_EPOCHS)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 

train_loss: 1.0922739505767822 

test_loss: 1.072731142838796 

train_accuracy: 0.0
Epoch: 2 

train_loss: 1.0273101329803467 

test_loss: 1.016386148929596 

train_accuracy: 0.0
Epoch: 3 

train_loss: 0.9613878130912781 

test_loss: 0.9927446953455608 

train_accuracy: 0.0


In [54]:
model_03_results

{'train_loss': [tensor(1.0923, grad_fn=<DivBackward0>),
  tensor(1.0273, grad_fn=<DivBackward0>),
  tensor(0.9614, grad_fn=<DivBackward0>)],
 'train_acc': [0.0, 0.0, 0.0],
 'test_loss': [1.072731142838796, 1.016386148929596, 0.9927446953455608],
 'test_acc': [0.41333333333333333, 0.4266666666666667, 0.4666666666666667]}

# All in 1 script - Train, Evaluate and Save the model

In [66]:
%%writefile modular_scripts/train.py

from prompt_toolkit import output

import os
import torch

from torchvision import transforms
from timeit import default_timer as timer

import data_setup, engine, model_builder, utils

NUM_EPOCHS = 3
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

data_transform = transforms.Compose([transforms.Resize((64,64)),
transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir = test_dir,
                                                                               transform = data_transform,
                                                                               batch_size=BATCH_SIZE)
model = model_builder.TinyVGG01(input_shape=3,
                                hidden_units=HIDDEN_UNITS,
                                output_shape=len(class_names))

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr = LEARNING_RATE)

engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS)
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_modular_scripts_tinyVGG.pth")

Overwriting modular_scripts/train.py


In [67]:
!python modular_scripts/train.py

Epoch: 1 

train_loss: 1.0988155603408813 

test_loss: 1.0887293418248494 

train_accuracy: 0.31640625
Epoch: 2 

train_loss: 1.092125415802002 

test_loss: 1.0609145561854045 

train_accuracy: 0.28515625
Epoch: 3 

train_loss: 1.0628067255020142 

test_loss: 1.0776806871096294 

train_accuracy: 0.41796875
Saving the model to models/05_modular_scripts_tinyVGG.pth
